# Mask and Social distancing Detection 

In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torchvision import transforms,models
import matplotlib.pyplot as plt
from PIL import Image
from random import shuffle
from bs4 import BeautifulSoup
from glob import iglob
from math import ceil
import cv2
import os
import torch

Data Augmentation

In [25]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [26]:
yolo_face = torch.hub.load("YOLO-FaceV2", "custom",path_or_model = 'best.pt', source="local")
yolo_face.eval()
# yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s',pretrained = True)
# yolo.classes = 0



                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1    656896  

Adding autoShape... 


autoShape(
  (model): Model(
    (model): Sequential(
      (0): Focus(
        (conv): Conv(
          (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU()
        )
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (cv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_st

In [69]:
tmp_tensor= yolo_face([cv2.resize(cv2.cvtColor(cv2.imread('images/maksssksksss158.png'),cv2.COLOR_BGR2RGB),(1000,1000))])
tmp_tensor.show()
#! example 157 158

# for idx,crop_list in enumerate(tmp_img):
#     tmp_list = []
#     for img in crop_list:
#         a = transform(Image.fromarray(img))
#         tmp_list.append(a)
#     pred = model(torch.stack(tmp_list))
#     label = pred.argmax(dim=1)
#     tmp_tensor[idx]['labels'] = label

/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [51]:
def yolo_dict(img_list):
    tensor_list = []
    crop_img_list = []
    # print('len of yolo_pred:',len(yolo_pred))
    for idx,img in enumerate(img_list):
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(1000,1000))
        yolo_pred = yolo_face(img)
        crop_img = []
        bboxes = yolo_pred.pandas().xyxy[0][['xmin','ymin','xmax','ymax']]
        score = yolo_pred.pandas().xyxy[0]['confidence']
        # print(type(bboxes))
        bbox_list = bboxes.to_numpy()
        score_list =score.to_numpy()
        try:
            bbox_tensor = torch.tensor(bbox_list)
            score_tensor = torch.tensor(score_list)
        except:
            bbox_tensor = torch.zeros([1,4])
            score_tensor = torch.zeros([1])
            print(idx)
        pred_tensor = {'boxes':bbox_tensor,'scores':score_tensor}
        tensor_list.append(pred_tensor)
        
        for bbox in bbox_list:
            # print(bbox)
            if bbox[1] == bbox[3]:
                # print('same')
                crop_img.append(img[ceil(bbox[1]):ceil(bbox[3])+1,ceil(bbox[0]):ceil(bbox[2])+1,:])
            else:
                # print(img.shape)
                c_i = img[ceil(bbox[1]):ceil(bbox[3]),ceil(bbox[0]):ceil(bbox[2]),:]
                # print(ceil(bbox[1]),ceil(bbox[3]),ceil(bbox[0]),ceil(bbox[2]))
                # print(c_i.shape)
                crop_img.append(c_i)
        # print(crop_img)
        crop_img_list.append(crop_img)
        # print('going to break')
    return tensor_list,crop_img_list

In [52]:
img_paths = ['images/maksssksksss163.png']
img_list = []
for img_path in img_paths:
    img_list.append(cv2.imread(img_path))
tmp_1, tmp_2 = yolo_dict(img_list)
print(tmp_1)
# yolo_pred = yolo(img_list)
# yolo_pred.pandas().xyxy[0]


[{'boxes': tensor([[522.18372, 357.19958, 563.37653, 412.08435],
        [412.55637, 391.93115, 471.91431, 475.10034],
        [974.39740, 356.66782, 996.26746, 386.38974],
        [272.74835, 387.66205, 302.17722, 424.43143]], dtype=torch.float64), 'scores': tensor([0.82286, 0.82057, 0.28425, 0.28393], dtype=torch.float64)}]


/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


### DataSet


In [53]:
#Load train and test set
train_dir = 'FaceMask_12k/Train'
test_dir = 'FaceMask_12k/Test'
val_dir = 'FaceMask_12k/Validation'
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [54]:
class MaskDataset(object):
    def __init__(self, transform,path):
        self.transform = transform
        # load all image files, sorting them to
        # ensure that they are aligned
        # path in forms of 'FaceMask_12k/Test/'
        self.path = path
        
        pos_imgs = []
        pos_imgs_path = list(sorted(os.listdir(path+'/WithMask')))
        for img_path in pos_imgs_path:
            pos_imgs.append(Image.open(path+'/WithMask/'+img_path))
            
        neg_imgs = []
        neg_imgs_path = list(sorted(os.listdir(path+'/WithoutMask')))
        for img_path in neg_imgs_path:
            neg_imgs.append(Image.open(path+'/WithoutMask/'+img_path))
        self.imgs = [*pos_imgs,*neg_imgs]
        
        pos_label = torch.ones(len(pos_imgs),dtype=torch.long)
        neg_label = torch.zeros(len(neg_imgs),dtype=torch.long)
        self.label = [*pos_label,*neg_label]
        
        self.data = list(zip(self.imgs,self.label))
        shuffle(self.data)
        
    def __getitem__(self, idx):
        img, label = self.data[idx]
        if self.transform is not None:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return len(self.imgs)

In [55]:
train_data = MaskDataset(transform,train_dir)
train_data = torch.utils.data.DataLoader(train_data,batch_size = 10)
val_data = MaskDataset(transform,val_dir)
val_data = torch.utils.data.DataLoader(val_data,batch_size = 10)

### Building ResNet 50 transfer learning model.

In [56]:
class ResNet_Trans(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet_50 = models.resnet50(pretrained=True)
        self.extractor = torch.nn.Sequential(*(list(self.resnet_50.children())[:-1]))
        self.sm = torch.nn.Softmax(dim=0)
        self.lr1 = torch.nn.Linear(2048,1024)
        self.lr2 = torch.nn.Linear(1024,2)
    
    def forward(self,x):
        x = self.extractor(x)
        x = x.transpose(1, 3)
        x = self.lr1(x)
#         print(x.shape)
        x = x.squeeze(dim = 1)
        x = x.squeeze(dim = 1)
        x = self.sm(x)
        x = self.lr2(x)
        return x
    
model = ResNet_Trans().to(device)
num_epochs = 0

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
loss_func = torch.nn.CrossEntropyLoss()

/opt/miniconda3/envs/image/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [57]:
for epoch in range(num_epochs):
    model.train()
    i = 0
    train_epoch_loss = 0
    val_epoch_loss = 0
    running_loss = 0
    for img, annotation in train_data:
        i += 1
        pred = model(img)
        print(pred,annotation)
        loss = loss_func(pred,annotation)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 20 == 0:
            print(
            f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}",
            end="\t",
        )
            running_loss = 0

Load Model State

In [58]:
model.load_state_dict(torch.load('model_weight.pt',map_location='cpu'))
model.eval()

ResNet_Trans(
  (resnet_50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [59]:
correct_pred = []
wrong_pred = []
count = 0
i=0
for img, annotation in val_data:
    i += 1
    val_pred = model(img)
    label = val_pred.argmax(dim=1)
    # print(torch.eq(label,annotation).tolist())
    a = torch.eq(label,annotation).tolist()
    if False not in torch.eq(label,annotation).tolist():
        correct_pred.append((label,annotation))
    else:
        count += a.count(False)
        wrong_pred.append((label,annotation))

In [60]:
print('Correct Prediction')
for label,annotation in correct_pred:
    print(label,annotation)
print('Wrong Prediction')
for label,annotation in wrong_pred:
    print(label,annotation)

Correct Prediction
tensor([1, 1, 0, 1, 1, 0, 0, 0, 1, 1]) tensor([1, 1, 0, 1, 1, 0, 0, 0, 1, 1])
tensor([0, 1, 1, 1, 0, 1, 1, 1, 1, 1]) tensor([0, 1, 1, 1, 0, 1, 1, 1, 1, 1])
tensor([1, 0, 0, 0, 1, 1, 0, 1, 1, 0]) tensor([1, 0, 0, 0, 1, 1, 0, 1, 1, 0])
tensor([0, 1, 1, 0, 1, 1, 0, 0, 1, 0]) tensor([0, 1, 1, 0, 1, 1, 0, 0, 1, 0])
tensor([1, 0, 0, 1, 0, 0, 1, 1, 1, 1]) tensor([1, 0, 0, 1, 0, 0, 1, 1, 1, 1])
tensor([1, 1, 0, 1, 0, 0, 1, 1, 0, 0]) tensor([1, 1, 0, 1, 0, 0, 1, 1, 0, 0])
tensor([0, 0, 0, 0, 1, 1, 1, 1, 0, 0]) tensor([0, 0, 0, 0, 1, 1, 1, 1, 0, 0])
tensor([0, 0, 1, 1, 0, 0, 1, 0, 0, 1]) tensor([0, 0, 1, 1, 0, 0, 1, 0, 0, 1])
tensor([1, 1, 0, 0, 0, 1, 1, 1, 0, 0]) tensor([1, 1, 0, 0, 0, 1, 1, 1, 0, 0])
tensor([1, 0, 1, 1, 1, 0, 0, 0, 0, 0]) tensor([1, 0, 1, 1, 1, 0, 0, 0, 0, 0])
tensor([0, 1, 1, 1, 0, 1, 0, 1, 0, 1]) tensor([0, 1, 1, 1, 0, 1, 0, 1, 0, 1])
tensor([0, 0, 0, 0, 1, 1, 1, 0, 0, 0]) tensor([0, 0, 0, 0, 1, 1, 1, 0, 0, 0])
tensor([1, 1, 0, 1, 1, 0, 0, 0, 0, 0]) tensor

In [61]:
img_paths = iglob('images/*')
img_list = []
for img_path in sorted(img_paths,key=lambda x: int(x.split(sep='maksssksksss')[1].split(sep='.')[0])):
    img_list.append(cv2.imread(img_path))
print(len(img_list))

# tmp = []
# for face in face_list:
#     face_tensor = transform(Image.fromarray(face,mode="RGB")).unsqueeze(dim=0)
#     face_pred = model(face_tensor)
#     label = face_pred.argmax(dim=1)
#     tmp.extend(label)
# pred_result = torch.tensor(tmp)


libpng warning: iCCP: Not recognizing known sRGB profile that has been edited


853


In [62]:
pred_dict, crop_img_list = yolo_dict(img_list)

/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn

15


/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn

112


/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn

...

/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn

811


/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


813


/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/miniconda3/envs/image/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn

In [63]:
print(len(crop_img_list[0]))
for img in crop_img_list[0]:
    print(img)

3
[[[ 69  67  76]
  [ 74  70  79]
  [ 80  75  84]
  ...
  [111  98 103]
  [110  97 104]
  [118 104 110]]

 [[ 70  67  76]
  [ 74  70  79]
  [ 79  73  83]
  ...
  [112  99 104]
  [112  99 105]
  [119 105 110]]

 [[ 71  68  77]
  [ 75  69  79]
  [ 78  71  81]
  ...
  [114 101 104]
  [113 101 105]
  [120 106 111]]

 ...

 [[ 65  48  50]
  [ 61  46  49]
  [ 57  43  45]
  ...
  [ 84  50  66]
  [ 88  54  71]
  [ 91  56  75]]

 [[ 67  49  52]
  [ 63  47  50]
  [ 57  44  46]
  ...
  [ 86  52  68]
  [ 90  56  72]
  [ 92  57  76]]

 [[ 69  50  53]
  [ 64  48  51]
  [ 58  45  47]
  ...
  [ 89  55  70]
  [ 92  58  74]
  [ 93  57  77]]]
[[[213 215 212]
  [219 221 217]
  [225 227 222]
  ...
  [ 10  10   9]
  [ 18  18  18]
  [ 42  42  42]]

 [[213 215 212]
  [225 226 222]
  [234 236 231]
  ...
  [ 19  19  19]
  [ 20  20  20]
  [ 27  27  27]]

 [[213 215 212]
  [230 232 228]
  [243 245 240]
  ...
  [ 29  29  28]
  [ 22  22  22]
  [ 12  12  13]]

 ...

 [[181 191 153]
  [169 179 142]
  [160 170 134]
  

In [64]:
# crop image 過 yolo
for idx,crop_list in enumerate(crop_img_list):
    # print(idx)
    tmp_list = []
    for img in crop_list:
        try:
            a = transform(Image.fromarray(img))
            tmp_list.append(a)
        except Exception as e:
            tmp_list.append(torch.zeros((3,244,244)))
    if len(tmp_list) > 0:
        pred = model(torch.stack(tmp_list))
        label = pred.argmax(dim=1)
        pred_dict[idx]['labels'] = label
    else:
        pred_dict[idx]['labels'] = torch.zeros((1))


In [65]:
def generate_box(obj,width,height):
    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text)
    xmax = int(obj.find('xmax').text)
    ymax = int(obj.find('ymax').text)
    xmin *= (1000/width)
    xmax *= (1000/width)
    ymin *= (1000/height)
    ymax *= (1000/height)
    return [xmin, ymin, xmax, ymax]

def generate_label(obj):
    if obj.find('name').text == "with_mask":
        return 1
    elif obj.find('name').text == "mask_weared_incorrect":
        return 1
    return 0

def generate_target(image_id, file): 
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, 'xml')
        objects = soup.find_all('object')
        width = int(soup.find('size').find('width').text)
        height = int(soup.find('size').find('height').text)
        num_objs = len(objects)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i,width,height))
            labels.append(generate_label(i))
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # Tensorise img_id
        img_id = torch.tensor([image_id])
        # Annotation is in dictionary format
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = img_id
        
        return target


In [66]:
class MaskDataset(object):
    def __init__(self, transforms):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir("images/")))
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        if idx >= 853:
            return False
        file_label = 'maksssksksss'+ str(idx) + '.xml'
        label_path = os.path.join("annotations/", file_label)
        target = generate_target(idx, label_path)
        return target

    def __len__(self):
        return len(self.imgs)

In [67]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
mAP = MeanAveragePrecision()
ground_truth = []
for i in range(len(pred_dict)):
    file_label = 'maksssksksss'+ str(i) + '.xml'
    label_path = os.path.join("annotations/", file_label)
    target = generate_target(i, label_path)
    ground_truth.append(target)
error_log = []
idx = 0
mAP.update(pred_dict,ground_truth)
print(mAP.compute())
# for annotations in ground_truth:
#     pred = pred_dict[idx]
#     idx += 1
#     # print('pred:',pred,'annotation',annotations)
#     # break
#     # bbox = 
#     try:
#         mAP.update([pred],[annotations])
#         # mAP.compute()
#     except Exception as e:
#         print(pred,'\n~~\n',annotations)
#         error_log.append((pred,annotation))
#         break


{'map': tensor(0.18272), 'map_50': tensor(0.51145), 'map_75': tensor(0.05793), 'map_small': tensor(0.04129), 'map_medium': tensor(0.20042), 'map_large': tensor(0.18317), 'mar_1': tensor(0.09861), 'mar_10': tensor(0.25589), 'mar_100': tensor(0.27841), 'mar_small': tensor(0.10932), 'mar_medium': tensor(0.30324), 'mar_large': tensor(0.28469), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.)}


In [68]:
# mAP = MeanAveragePrecision()
# print(tmp_tensor,'\n\n',ground_truth[2])
# mAP.update(tmp_tensor,[ground_truth[2]])
mAP.compute()

{'map': tensor(0.18272),
 'map_50': tensor(0.51145),
 'map_75': tensor(0.05793),
 'map_small': tensor(0.04129),
 'map_medium': tensor(0.20042),
 'map_large': tensor(0.18317),
 'mar_1': tensor(0.09861),
 'mar_10': tensor(0.25589),
 'mar_100': tensor(0.27841),
 'mar_small': tensor(0.10932),
 'mar_medium': tensor(0.30324),
 'mar_large': tensor(0.28469),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}